In [ ]:
import itertools
import os

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import pandas as pd
from scipy import spatial
from scipy import interpolate
import seaborn as sns

from utils import (estimate_normals_pca, estimate_normals_spline,
                   orient_normals, n2rgb)
from plotting import set_axes_equal, set_defense_context, draw_unit_cube

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
c = sns.color_palette('rocket', n_colors=1)

In [ ]:
def add_coordinate_frame(ax):
    """Set RGB coordinate frame to axes.
    
    Parameters
    ----------
    ax : matplotlib.axes._subplots.Axes3DSubplot
        3-D axes subplot.
    
    Returns
    -------
    matplotlib.axes._subplots.Axes3DSubplot
        Axes with coordinate frame.
    """
    ax.quiver(-1.5, -1, -1.5, 0.75, 0, 0, lw=0.5, color='r')
    ax.text(0, -1, -1.5, s='$x$', color='r', fontweight='bold')
    ax.quiver(-1.5, -1, -1.5, 0, 0.75, 0, lw=0.5, color='g')
    ax.text(-1.5, +0.25, -1.5, s='$y$', color='g', fontweight='bold')
    ax.quiver(-1.5, -1, -1.5, 0, 0, 0.75, lw=0.5, color='b')
    ax.text(-1.5, -1, -0.5, s='$z$', color='b', fontweight='bold')
    ax.scatter(-1.5, -1, -1.5, s=7, color='k', depthshade=False)
    return ax

# PCA via Open3D - unit normal

In [ ]:
# load ear coordinates

xyz = pd.read_csv(os.path.join('data', 'ear.xyz')).values * 100  # in cm

In [ ]:
# estimate unit normals

n_pca = estimate_normals_pca(xyz, knn=111)
n_pca = orient_normals(xyz, n_pca, knn=30)

In [ ]:
# grid downsample

N = xyz.shape[0]
num = 0.05 * N
mask = np.arange(0, N, int(N/num))

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(4, 4), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    ax.quiver(*xyz[mask].T, *n_pca[mask].T, color='k',
              length=0.55, lw=0.5)
    ax.view_init(20, 150)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    plt.show()
    fname = 'normals.png'
    fig.savefig(os.path.join('figures', 'ear', fname),
                dpi=500, bbox_inches='tight')

In [ ]:
# convert arrows to rgb cube

with set_defense_context():
    fig = plt.figure(figsize=(4, 4), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    s = ax.scatter(*xyz.T, color=n2rgb(n_pca), s=0.3)
    ax = add_coordinate_frame(ax)
    ax.view_init(20, 155)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    plt.show()
    fname = 'normals-rgb.png'
    fig.savefig(os.path.join('figures', 'ear', fname),
                dpi=500, bbox_inches='tight')

# Bivariate B-spline - surface (curvature) normal

In [ ]:
# estimate surface normals

n_spl = estimate_normals_spline(xyz, unit=True, knn=111)
n_spl = orient_normals(xyz, n_spl, knn=30)

In [ ]:
# convert arrows to rgb cube

with set_defense_context():
    fig = plt.figure(figsize=(4, 4), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    s = ax.scatter(*xyz.T, color=n2rgb(n_spl), s=0.3)
    ax = add_coordinate_frame(ax)
    ax.view_init(20, 155)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    plt.show()
    fname = 'surface-normals.png'
    fig.savefig(os.path.join('figures', 'ear', fname),
                dpi=500, bbox_inches='tight')

# Error angle

In [ ]:
# compute error

theta = np.arccos(
    np.round(  # to avoid numerical instabilities
        np.sum(n_pca * n_spl, axis=1),  # scalar product
    6)
) * 180 / np.pi
theta_rms = np.sqrt(np.mean(theta) ** 2)
theta_rms

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(4, 3))
    ax = plt.axes()
    ax.hist(theta, bins=31, density=True, ec='k', fc=c[0])
    ax.set(xlabel=r'$\theta$ (°)', ylabel='density',
           title=f'rms($\\theta$) = {theta_rms:.2f}°')
    fig.tight_layout()
    sns.despine()
    plt.show()
    fname = 'error-hist.png'
    fig.savefig(os.path.join('figures', 'ear', fname),
                dpi=500, bbox_inches='tight')

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(4, 4), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    s = ax.scatter(*xyz.T, c=theta, s=0.3)
    ax = add_coordinate_frame(ax)
    ax.view_init(20, 155)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    fig.colorbar(s, ax=ax, orientation='vertical', shrink=0.5, pad=-0.25,
                 label=r'$\theta$ (°)')
    plt.show()
    fname = 'error-dist.png'
    fig.savefig(os.path.join('figures', 'ear', fname),
                dpi=500, bbox_inches='tight')

# RGB cube

In [ ]:
# set up coloring for rgb cube

pts = np.array(list(itertools.product([0, 1], repeat=3)))
cs = ['black', 'blue', 'green', 'cyan', 'red', 'magenta', 'yellow', 'white']
pairs = pd.DataFrame(data=pts, columns=['x', 'y', 'z'])
pairs['cs'] = cs

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(2, 2), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    ax = draw_unit_cube(ax)
    ax.scatter(*pts.T, c=cs, edgecolor='k', depthshade=False, s=500)
    ax.view_init(20, 155)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    fig.tight_layout()
    plt.show()
    fname = 'rgb-cube.png'
    fig.savefig(os.path.join('figures', 'ear', fname),
                dpi=500, bbox_inches='tight')